In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix

In [2]:
image_dir = Path('C:/Users/hp/Documents/Miniproject/tomato/train')

In [3]:
#create file dataframe
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths,labels], axis=1)

In [4]:
image_df

,Filepath,Label
0,C:\Users\hp\Documents\Miniproject\tomato\train...,Tomato___Bacterial_spot
1,C:\Users\hp\Documents\Miniproject\tomato\train...,Tomato___Bacterial_spot
2,C:\Users\hp\Documents\Miniproject\tomato\train...,Tomato___Bacterial_spot
3,C:\Users\hp\Documents\Miniproject\tomato\train...,Tomato___Bacterial_spot
4,C:\Users\hp\Documents\Miniproject\tomato\train...,Tomato___Bacterial_spot
...,...,...
4774,C:\Users\hp\Documents\Miniproject\tomato\train...,Tomato___Tomato_Yellow_Leaf_Curl_Virus
4775,C:\Users\hp\Documents\Miniproject\tomato\train...,Tomato___Tomato_Yellow_Leaf_Curl_Virus
4776,C:\Users\hp\Documents\Miniproject\tomato\train...,Tomato___Tomato_Yellow_Leaf_Curl_Virus
4777,C:\Users\hp\Documents\Miniproject\tomato\train...,Tomato___Tomato_Yellow_Leaf_Curl_Virus


In [6]:
image_df['Label'].unique()

array(['Tomato___Bacterial_spot', 'Tomato___Early_blight',
       'Tomato___healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold',
       'Tomato___Septoria_leaf_spot',
       'Tomato___Spider_mites Two-spotted_spider_mite',
       'Tomato___Target_Spot', 'Tomato___Tomato_mosaic_virus',
       'Tomato___Tomato_Yellow_Leaf_Curl_Virus'], dtype=object)

In [5]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [7]:
train_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(244,244),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)#class_mode = 'binary' ---> for two classes

val_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(244,244),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)#class_mode = 'binary' ---> for two classes

test_images = test_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(244,244),
    color_mode='rgb',
    class_mode='sparse',
    batch_size=32,
    shuffle=False
)#class_mode = 'binary' ---> for two classes

Found 2676 validated image filenames belonging to 10 classes.
Found 669 validated image filenames belonging to 10 classes.
Found 3345 validated image filenames belonging to 10 classes.


In [34]:
#Training

inputs = tf.keras.Input(shape=(256,256,3))
x = tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)


x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(10,activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs = outputs)



In [35]:
x

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense_24')>

In [36]:
model.compile(optimizer = 'adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy']
             )


In [37]:
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_24 (Conv2D)          (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 127, 127, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 62, 62, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 60, 60, 64)        1849

In [38]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        patience=3
        )
    ]
)

Epoch 1/10
84/84 [==============================] - 171s 2s/step - loss: 2.2345 - accuracy: 0.1420 - val_loss: 2.0811 - val_accuracy: 0.1973 - lr: 0.0010
Epoch 2/10
84/84 [==============================] - 164s 2s/step - loss: 1.9761 - accuracy: 0.2425 - val_loss: 1.9145 - val_accuracy: 0.3124 - lr: 0.0010
Epoch 3/10
84/84 [==============================] - 165s 2s/step - loss: 1.6930 - accuracy: 0.3763 - val_loss: 1.5630 - val_accuracy: 0.4365 - lr: 0.0010
Epoch 4/10
84/84 [==============================] - 160s 2s/step - loss: 1.5463 - accuracy: 0.4268 - val_loss: 1.6739 - val_accuracy: 0.3812 - lr: 0.0010
Epoch 5/10
84/84 [==============================] - 155s 2s/step - loss: 1.4082 - accuracy: 0.4851 - val_loss: 1.5052 - val_accuracy: 0.4230 - lr: 0.0010
Epoch 6/10
84/84 [==============================] - 162s 2s/step - loss: 1.3397 - accuracy: 0.5153 - val_loss: 1.7638 - val_accuracy: 0.4036 - lr: 0.0010
Epoch 7/10
84/84 [==============================] - 163s 2s/step - loss: 1.3

In [10]:
x

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense_1')>

In [11]:
results = model.evaluate(test_images, verbose=0)
print('Test loss: {:.5f}'.format(results[0]))
print('Test accuracy: {:.2f}'.format(results[1]*100))

Test loss: 1.32533
Test accuracy: 49.69


In [ ]:
predictions = (model.predict(test_images) >= 0.5).astype(np.int)

cm = confusion_matrix (test_images.labels,predictions,labels=[0, 1])
clr = classification_report(test_images.labels,predictions,labels=[0, 1], target_names = ['Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___healthy','Tomato___Late_blight','Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite','Tomato___Target_Spot','Tomato___Tomato_mosaic_virus','Tomato___Tomato_Yellow_Leaf_Curl_Virus'])

plt.figure(figsize=(6,6))
sns.heatmap(cm,annot=True,fmt='g',vmin=0,cmap='Blues',cbar=False)
plt.xticks(ticks=[0.5, 1.5],labels=['Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___healthy','Tomato___Late_blight','Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite','Tomato___Target_Spot','Tomato___Tomato_mosaic_virus','Tomato___Tomato_Yellow_Leaf_Curl_Virus'])
plt.xticks(ticks=[0.5, 1.5],labels=['Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___healthy','Tomato___Late_blight','Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite','Tomato___Target_Spot','Tomato___Tomato_mosaic_virus','Tomato___Tomato_Yellow_Leaf_Curl_Virus'])
plt.xlabel("predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:\n--------------------------------\n",clr)


In [12]:
import pickle

In [13]:
pickle.dump(model, open('model_tomato.pkl','wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\conv2d_2
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\global_average_pooling2d
......vars
...layers\input_layer
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...layers\max_pooling2d_2
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........23
.........24
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras 

In [14]:
image_df.to_csv('TomatoImages.csv')

In [15]:
model.predict(test_images[1])

ValueError: in user code:

    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 216, in assert_input_compatibility
        raise ValueError(

    ValueError: Layer "model" expects

In [16]:
test_images[1]

(array([[[[0.5647059 , 0.5411765 , 0.54901963],
          [0.49411768, 0.47058827, 0.4784314 ],
          [0.5411765 , 0.5176471 , 0.5254902 ],
          ...,
          [0.5647059 , 0.5176471 , 0.53333336],
          [0.41960788, 0.37254903, 0.38823533],
          [0.50980395, 0.46274513, 0.4784314 ]],
 
         [[0.5568628 , 0.53333336, 0.5411765 ],
          [0.5294118 , 0.5058824 , 0.5137255 ],
          [0.5921569 , 0.5686275 , 0.5764706 ],
          ...,
          [0.48235297, 0.43529415, 0.45098042],
          [0.46274513, 0.4156863 , 0.43137258],
          [0.54509807, 0.49803925, 0.5137255 ]],
 
         [[0.56078434, 0.5372549 , 0.54509807],
          [0.5176471 , 0.49411768, 0.5019608 ],
          [0.54509807, 0.52156866, 0.5294118 ],
          ...,
          [0.56078434, 0.5137255 , 0.5294118 ],
          [0.5254902 , 0.4784314 , 0.49411768],
          [0.45098042, 0.4039216 , 0.41960788]],
 
         ...,
 
         [[0.4784314 , 0.43529415, 0.45098042],
          [0.51764

In [32]:
pred = model.predict(train_df.iloc[4,0])

IndexError: tuple index out of range

In [28]:
print(train_df.iloc[4,0])

C:\Users\hp\Documents\Miniproject\tomato\train\Tomato___Tomato_Yellow_Leaf_Curl_Virus\f40054b6-9432-4a53-994a-d614840b8540___UF.GRC_YLCV_Lab 02544.JPG


In [36]:
import cv2

In [37]:
img = plt.imread("C:/Users/hp/Documents/Miniproject/tomato/train/Tomato___Early_blight/004cbe60-8ff9-4965-92df-e86694d5e9ba___RS_Erly.B 8253.JPG")
img.shape

(256, 256, 3)

In [38]:
p = model.predict(img)

ValueError: in user code:

    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\hp\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "mod